In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#config

In [3]:
SOURCE_FILE = ""
OUTPUT_DIR = ""
HORIZON = 90

#benchmark

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
import holidays

# --- 1. Data Loading and Initial Exploration ---
print("--- Starting Data Loading and Exploration ---")

# Load the dataset
try:
    # MODIFICATION: Added encoding='utf-16' to handle files saved from Excel or other sources.
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")
    print("Initial data preview:")
    print(df.head())
    print("\nData Info:")
    df.info()
except FileNotFoundError:
    print("Error: 'Phone ASA + SL (table).csv' not found. Please upload the file to your Colab environment.")
    exit()


# --- 2. Data Cleaning and Preprocessing ---
print("\n--- Starting Data Cleaning and Preprocessing ---")

# Standardize column names (e.g., remove spaces, convert to lowercase)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# MODIFICATION: Focus only on the essential columns first
df = df[['date', 'answered_sessions']]

# Convert 'date' to datetime objects
df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')

# MODIFICATION: Clean the target variable 'answered_sessions'
# It's an object type with commas and quotes, so we convert it to a numeric type.
if df['answered_sessions'].dtype == 'object':
    df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)


print("\nCleaned data preview (focused on target and date):")
print(df.head())
print("\nData Info after cleaning:")
df.info()

# Set 'date' as the index for time series analysis
df = df.set_index('date').sort_index()


# --- 3. Feature Engineering ---
print("\n--- Starting Feature Engineering ---")

def create_features(dataframe):
    """
    Creates time series features from a datetime index.
    """
    dataframe['dayofweek'] = dataframe.index.dayofweek
    dataframe['quarter'] = dataframe.index.quarter
    dataframe['month'] = dataframe.index.month
    dataframe['year'] = dataframe.index.year
    dataframe['dayofyear'] = dataframe.index.dayofyear
    dataframe['dayofmonth'] = dataframe.index.day
    dataframe['weekofyear'] = dataframe.index.isocalendar().week.astype(int)
    return dataframe

df = create_features(df)

# Add holiday features
us_holidays = holidays.US()
df['is_holiday'] = df.index.map(lambda x: 1 if x in us_holidays else 0)

print("\nData with new features:")
print(df.head())


# --- 4. Exploratory Data Analysis (EDA) ---
print("\n--- Starting Exploratory Data Analysis (EDA) ---")

# Plot the target variable over time
plt.figure(figsize=(15, 5))
plt.plot(df.index, df['answered_sessions'], label='Answered Sessions')
plt.title('Answered Sessions Over Time')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.legend()
plt.grid(True)
plt.show()

# Visualize seasonality
fig, axes = plt.subplots(2, 1, figsize=(14, 12))
sns.boxplot(x='dayofweek', y='answered_sessions', data=df, ax=axes[0])
axes[0].set_title('Volume by Day of the Week')
axes[0].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

sns.boxplot(x='month', y='answered_sessions', data=df, ax=axes[1])
axes[1].set_title('Volume by Month')
plt.tight_layout()
plt.show()

# --- 5. Model Training and Evaluation ---
print("\n--- Starting Model Training and Evaluation ---")

# Define features (X) and target (y)
TARGET = 'answered_sessions'
# MODIFICATION: We will only use time-based features for now as requested.
# The other metrics can be added back later if they prove to be useful regressors.
FEATURES = ['dayofweek', 'quarter', 'month', 'year',
            'dayofyear', 'dayofmonth', 'weekofyear', 'is_holiday']

# Handle potential missing values in regressors (e.g., fill with median)
for col in FEATURES:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())


X = df[FEATURES]
y = df[TARGET]

# Split data into training and testing sets
# We use a simple time-based split for this example
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero by handling zero values in y_true
    return np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, np.finfo(np.float64).eps))) * 100

# Function to calculate sMAPE
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.abs(y_pred - y_true) / np.maximum(denominator, np.finfo(np.float64).eps)) * 100


# Function to evaluate models
def evaluate_model(true_values, predictions, model_name):
    mae = mean_absolute_error(true_values, predictions)
    mse = mean_squared_error(true_values, predictions)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(true_values, predictions)
    smape = symmetric_mean_absolute_percentage_error(true_values, predictions)


    print(f"--- {model_name} Evaluation ---")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"sMAPE: {smape:.2f}%")
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape, 'sMAPE': smape}

# --- Model 1: XGBoost ---
print("\n--- Training XGBoost Model ---")
# MODIFICATION: Removed eval_set and verbose from the model constructor.
reg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50,
                       objective='reg:squarederror')
# MODIFICATION: Passed eval_set and verbose directly to the .fit() method.
reg.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False)

# Feature importance
feature_imp = pd.DataFrame(data=reg.feature_importances_,
                           index=reg.feature_names_in_,
                           columns=['importance'])
feature_imp.sort_values('importance').plot(kind='barh', title='XGBoost Feature Importance')
plt.show()

# Prediction and evaluation
xgb_preds = reg.predict(X_test)
xgb_metrics = evaluate_model(y_test, xgb_preds, "XGBoost")


# --- Model 2: LightGBM ---
print("\n--- Training LightGBM Model ---")
lgb_reg = lgb.LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=31)
lgb_reg.fit(X_train, y_train)
lgb_preds = lgb_reg.predict(X_test)
lgb_metrics = evaluate_model(y_test, lgb_preds, "LightGBM")

# --- Model 3: Random Forest ---
print("\n--- Training Random Forest Model ---")
rf_reg = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1, max_depth=10)
rf_reg.fit(X_train, y_train)
rf_preds = rf_reg.predict(X_test)
rf_metrics = evaluate_model(y_test, rf_preds, "Random Forest")


# --- Model 4: Facebook Prophet ---
print("\n--- Training Prophet Model ---")
# Prophet requires a specific dataframe format: 'ds' for dates and 'y' for values
prophet_df = df.reset_index()[['date', 'answered_sessions']].rename(columns={'date': 'ds', 'answered_sessions': 'y'})

# Add holidays to Prophet
holidays_df = pd.DataFrame({
    'holiday': 'us_holiday',
    'ds': df[df['is_holiday'] == 1].index,
    'lower_window': 0,
    'upper_window': 1,
})

# Split for Prophet
prophet_train = prophet_df.iloc[:train_size]
prophet_test = prophet_df.iloc[train_size:]

# Initialize and fit the model
model = Prophet(holidays=holidays_df)
model.fit(prophet_train)

# Create future dataframe for predictions
future = model.make_future_dataframe(periods=len(prophet_test), freq='D')
forecast = model.predict(future)

# Extract predictions for the test period
prophet_preds = forecast.iloc[-len(prophet_test):]['yhat']
prophet_metrics = evaluate_model(y_test, prophet_preds, "Prophet")

# Plot Prophet forecast
fig1 = model.plot(forecast)
plt.title('Prophet Forecast')
plt.show()
fig2 = model.plot_components(forecast)
plt.show()


# --- 6. Model Comparison ---
print("\n--- Final Model Comparison ---")

# Visualize all model predictions against actuals
plt.figure(figsize=(15, 6))
plt.plot(y_test.index, y_test, label='Actual Values', color='blue', linewidth=2)
plt.plot(y_test.index, xgb_preds, label='XGBoost Preds', color='orange', linestyle='--')
plt.plot(y_test.index, lgb_preds, label='LightGBM Preds', color='green', linestyle='--')
plt.plot(y_test.index, rf_preds, label='Random Forest Preds', color='red', linestyle='--')
plt.plot(y_test.index, prophet_preds, label='Prophet Preds', color='purple', linestyle='--')
plt.title('Comparison of Model Forecasts')
plt.xlabel('Date')
plt.ylabel('Answered Sessions')
plt.legend()
plt.grid(True)
plt.show()

# Compare metrics
metrics_df = pd.DataFrame({
    'XGBoost': xgb_metrics,
    'LightGBM': lgb_metrics,
    'Random Forest': rf_metrics,
    'Prophet': prophet_metrics
}).T

print("\nModel Performance Metrics:")
print(metrics_df)

print("\n--- Forecasting Module Complete ---")

--- Starting Data Loading and Exploration ---
Error: 'Phone ASA + SL (table).csv' not found. Please upload the file to your Colab environment.

--- Starting Data Cleaning and Preprocessing ---


NameError: name 'df' is not defined

#random forest benchmark

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import holidays
import plotly.graph_objects as go
import plotly.express as px
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("--- Starting Random Forest Forecasting Module ---")

# --- 1. Symmetric Mean Absolute Percentage Error (sMAPE) Function ---
def smape(y_true, y_pred):
    """
    Calculates the Symmetric Mean Absolute Percentage Error (sMAPE).
    sMAPE is a robust metric for percentage errors, especially when actuals can be zero.
    """
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    # Handle the case where both true and predicted are zero
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

# --- 2. Data Loading and Preprocessing ---
print("\n--- Loading and Preprocessing Data ---")
try:
    # Load the dataset using the correct encoding and settings
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")

    # Standardize column names for consistency
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

    # Select only the columns we need for this forecast
    df = df[['date', 'answered_sessions']]

    # Clean and format the data
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)

    # Set the date as the index and sort the data chronologically
    df = df.set_index('date').sort_index()
    print("Data preprocessing complete.")

except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    exit() # Exit the script if the data file isn't available

# --- 3. Feature Engineering ---
def create_features(df):
    """
    Create time-series features from the datetime index.
    """
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    # Add a feature for US holidays
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    return df

df = create_features(df)
print("Feature engineering complete.")

# --- 4. Train/Test Split ---
# We split the data chronologically to simulate a real-world forecasting scenario.
split_date = df.index.max() - pd.Timedelta(days=45)
train = df.loc[df.index <= split_date]
test = df.loc[df.index > split_date]

TARGET = 'answered_sessions'
FEATURES = [col for col in df.columns if col != TARGET]

X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

# --- 5. Model Training ---
print("\n--- Training Random Forest Model ---")
# Initialize the Random Forest Regressor with common-sense parameters.
# n_estimators is the number of trees in the forest.
model = RandomForestRegressor(n_estimators=1000,
                              random_state=42,
                              n_jobs=-1) # Use all available CPU cores

# Fit the model to the training data
model.fit(X_train, y_train)
print("Model training complete.")

# --- 6. Model Evaluation ---
print("\n--- Evaluating Random Forest Model ---")
# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate performance metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
smape_score = smape(y_test, predictions)

# Display the results
print(f"MAE (Mean Absolute Error): {mae:.2f}")
print(f"MSE (Mean Squared Error): {mse:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"sMAPE (Symmetric MAPE): {smape_score:.2f}%")

# --- 7. Visualization ---

# Plot Feature Importance
print("\n--- Visualizing Feature Importance ---")
feature_imp = pd.DataFrame(data=model.feature_importances_,
                           index=model.feature_names_in_,
                           columns=['Importance'])

fig_imp = px.bar(feature_imp.sort_values('Importance'),
                 orientation='h',
                 title='Random Forest Feature Importance')
fig_imp.show()

# Plot Predictions vs. Actuals
print("\n--- Visualizing Predictions vs. Actuals ---")
test['prediction'] = predictions
fig_pred = go.Figure()
fig_pred.add_trace(go.Scatter(x=test.index, y=test[TARGET], mode='lines', name='Actual Values', line=dict(color='royalblue')))
fig_pred.add_trace(go.Scatter(x=test.index, y=test['prediction'], mode='lines', name='Predictions', line=dict(color='darkorange', dash='dash')))
fig_pred.update_layout(title='Random Forest: Predictions vs. Actual Values on Test Set',
                       xaxis_title='Date', yaxis_title='Answered Sessions')
fig_pred.show()


print("\n--- Random Forest Forecasting Module Complete ---")


--- Starting Random Forest Forecasting Module ---

--- Loading and Preprocessing Data ---
Error: The file 'Phone ASA + SL (table).csv' was not found.


NameError: name 'df' is not defined

#xgboost

In [ ]:
import pandas as pd
import numpy as np
import holidays
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("--- Starting Forecasting Visualization ---")

# --- 1. Load and Prepare Data (Similar to the previous script) ---
try:
    # Load the dataset using the correct encoding
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")

    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

    # Focus on the essential columns
    df = df[['date', 'answered_sessions']]

    # Convert 'date' to datetime and clean the target variable
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)

    # Set 'date' as the index
    df = df.set_index('date').sort_index()
    print("Data preparation complete.")

except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    print("Please make sure the file is uploaded to your Colab environment or Google Drive.")
    # Exit if the file is not found
    exit()


# --- 2. Feature Engineering Function ---
def create_features(df):
    """
    Create time series features based on the date index.
    """
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    # Add holiday feature
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    return df

df = create_features(df)
print("Feature engineering complete.")

# --- 3. Retrain Winning Model on All Data ---
print("\n--- Retraining Random Forest Model on the Entire Dataset ---")

TARGET = 'answered_sessions'
FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'is_holiday']

X_all = df[FEATURES]
y_all = df[TARGET]

# Initialize and train the final model
final_model = RandomForestRegressor(n_estimators=1000, max_features=0.8, random_state=42, n_jobs=-1)
final_model.fit(X_all, y_all)
print("Final model training complete.")

# --- 4. Create Future Forecast ---
print("\n--- Generating Future Forecast ---")

# Define the forecast period (e.g., 90 days into the future)
last_date = df.index.max()
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=90, freq='D')

# Create a future dataframe with the same features
future_df = pd.DataFrame(index=future_dates)
future_df = create_features(future_df)

# Generate predictions
future_predictions = final_model.predict(future_df[FEATURES])
future_df['forecast'] = future_predictions
print(f"Forecast generated for the next {len(future_df)} days.")
print("Forecast preview:")
print(future_df.head())


# --- 5. Visualize the Forecast ---
print("\n--- Creating Forecast Visualization ---")

# Create the plot
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(x=df.index, y=df[TARGET], mode='lines', name='Historical Data', line=dict(color='royalblue')))

# Add future forecast
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['forecast'], mode='lines', name='Future Forecast', line=dict(color='darkorange', dash='dash')))

# Update layout for a professional look
fig.update_layout(
    title='Phone Call Volume Forecast vs. Historical Data',
    xaxis_title='Date',
    yaxis_title='Answered Sessions (Volume)',
    template='plotly_white',
    legend=dict(x=0.01, y=0.98),
    font=dict(family="Arial, sans-serif", size=12)
)
# Define the path to save the CSV file
output_path = 'gdrive/My Drive/Gusto Performance/sources/preprocess/xgboost_forecast.csv'

# Export the future_df to a CSV file
future_df.to_csv(output_path)

print(f"Future forecast exported to {output_path}")
# Show the plot
fig.show()

print("\n--- Forecasting Visualization Complete ---")


--- Starting Forecasting Visualization ---
Dataset loaded successfully.
Data preparation complete.
Feature engineering complete.

--- Retraining Random Forest Model on the Entire Dataset ---
Final model training complete.

--- Generating Future Forecast ---
Forecast generated for the next 90 days.
Forecast preview:
            dayofweek  quarter  month  year  dayofyear  dayofmonth  \
2025-08-25          0        3      8  2025        237          25   
2025-08-26          1        3      8  2025        238          26   
2025-08-27          2        3      8  2025        239          27   
2025-08-28          3        3      8  2025        240          28   
2025-08-29          4        3      8  2025        241          29   

            weekofyear  is_holiday  forecast  
2025-08-25          35           0  2231.407  
2025-08-26          35           0  2001.456  
2025-08-27          35           0  2171.056  
2025-08-28          35           0  2186.648  
2025-08-29          35     


--- Forecasting Visualization Complete ---


#random forest

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import holidays
import plotly.graph_objects as go
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("--- Starting Final Random Forest Forecasting Script ---")

# --- 1. Data Loading and Preprocessing ---
print("\n--- Loading and Preprocessing Data ---")
try:
    # Load the dataset using the correct encoding and settings
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")

    # Standardize column names for consistency
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

    # Select only the columns we need for this forecast
    df = df[['date', 'answered_sessions']]

    # Clean and format the data
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)

    # Set the date as the index and sort the data chronologically
    df = df.set_index('date').sort_index()
    print("Data preprocessing complete.")

except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    exit() # Exit the script if the data file isn't available

# --- 2. Feature Engineering ---
def create_features(df):
    """
    Create time-series features from the datetime index.
    """
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    # Add a feature for US holidays
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    return df

df = create_features(df)
print("Feature engineering complete.")


# --- 3. Retrain Model on ALL Data ---
print("\n--- Retraining Random Forest Model on the Entire Dataset ---")
# To get the best forecast, we train the model on all available historical data.
TARGET = 'answered_sessions'
FEATURES = [col for col in df.columns if col != TARGET]

X_all = df[FEATURES]
y_all = df[TARGET]

# Initialize and train the final model
final_model = RandomForestRegressor(n_estimators=1000,
                                    random_state=42,
                                    n_jobs=-1) # Use all available CPU cores
final_model.fit(X_all, y_all)
print("Final model training complete.")


# --- 4. Generate 90-Day Future Forecast ---
print("\n--- Generating 90-Day Future Forecast ---")

# Determine the last date in our historical data
last_date = df.index.max()

# Create a new DataFrame for the next 90 days
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=90, freq='D')
future_df = pd.DataFrame(index=future_dates)

# Apply the same feature engineering to our future dates
future_df = create_features(future_df)

# Use our trained model to make predictions
future_predictions = final_model.predict(future_df[FEATURES])
future_df['forecast'] = future_predictions

print(f"Forecast generated for the next 90 days, from {future_df.index.min().date()} to {future_df.index.max().date()}.")


# --- 5. Visualize the Historical Data and Forecast ---
print("\n--- Visualizing Full History and 90-Day Forecast ---")

fig = go.Figure()

# Add the historical data trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df[TARGET],
                         mode='lines',
                         name='Historical Data',
                         line=dict(color='royalblue')))

# Add the future forecast trace
fig.add_trace(go.Scatter(x=future_df.index,
                         y=future_df['forecast'],
                         mode='lines',
                         name='90-Day Forecast',
                         line=dict(color='darkorange', dash='dash')))

# Update the layout for a professional look
fig.update_layout(
    title='Historical Call Volume and 90-Day Forecast',
    xaxis_title='Date',
    yaxis_title='Answered Sessions (Volume)',
    template='plotly_white',
    legend=dict(x=0.01, y=0.98),
    font=dict(family="Arial, sans-serif", size=12)
)
# Define the path to save the CSV file
output_path = 'gdrive/My Drive/Gusto Performance/sources/preprocess/rf_forecast.csv'

# Export the future_df to a CSV file
future_df.to_csv(output_path)

print(f"Future forecast exported to {output_path}")
fig.show()

print("\n--- Forecasting Script Complete ---")


--- Starting Final Random Forest Forecasting Script ---

--- Loading and Preprocessing Data ---
Dataset loaded successfully.
Data preprocessing complete.
Feature engineering complete.

--- Retraining Random Forest Model on the Entire Dataset ---
Final model training complete.

--- Generating 90-Day Future Forecast ---
Forecast generated for the next 90 days, from 2025-08-25 to 2025-11-22.

--- Visualizing Full History and 90-Day Forecast ---
Future forecast exported to gdrive/My Drive/Gusto Performance/sources/preprocess/rf_forecast.csv



--- Forecasting Script Complete ---


#xgboost param grid

In [ ]:
import pandas as pd
import numpy as np
import holidays
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("--- Starting Tuned Forecasting Visualization ---")

# --- 1. Load and Prepare Data ---
try:
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")

    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df = df[['date', 'answered_sessions']]
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)

    df = df.set_index('date').sort_index()
    print("Data preparation complete.")

except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    exit()

# --- 2. Feature Engineering ---
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    return df

df = create_features(df)
print("Feature engineering complete.")

# --- 3. Hyperparameter Tuning with GridSearchCV ---
print("\n--- Starting Hyperparameter Tuning for Random Forest ---")

TARGET = 'answered_sessions'
FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'is_holiday']

X_all = df[FEATURES]
y_all = df[TARGET]

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 0.8],
    'max_depth': [10, 20, None],
    'min_samples_leaf': [1, 2, 4]
}

# Use TimeSeriesSplit for cross-validation on time series data
tscv = TimeSeriesSplit(n_splits=5)

# Initialize the GridSearchCV object
# We use 'neg_mean_squared_error' because grid search aims to maximize a score,
# so we use the negative of a common error metric.
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_mean_squared_error',
    verbose=1
)

# Fit the grid search to the data
grid_search.fit(X_all, y_all)

# Get the best model and its parameters
best_rf_model = grid_search.best_estimator_
print("\n--- Hyperparameter Tuning Complete ---")
print("Best Parameters Found:")
print(grid_search.best_params_)


# --- 4. Create Future Forecast with the Tuned Model ---
print("\n--- Generating Future Forecast with Tuned Model ---")

last_date = df.index.max()
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=90, freq='D')

future_df = pd.DataFrame(index=future_dates)
future_df = create_features(future_df)

# Generate predictions using the best model found
future_predictions = best_rf_model.predict(future_df[FEATURES])
future_df['forecast'] = future_predictions
print(f"Forecast generated for the next {len(future_df)} days.")


# --- 5. Visualize the Forecast ---
print("\n--- Creating Forecast Visualization ---")

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df[TARGET], mode='lines', name='Historical Data', line=dict(color='royalblue')))
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['forecast'], mode='lines', name='Tuned Future Forecast', line=dict(color='darkorange', dash='dash')))

fig.update_layout(
    title='Phone Call Volume Forecast (Tuned Model)',
    xaxis_title='Date',
    yaxis_title='Answered Sessions (Volume)',
    template='plotly_white',
    legend=dict(x=0.01, y=0.98),
    font=dict(family="Arial, sans-serif", size=12)
)
# Define the path to save the CSV file
output_path = 'gdrive/My Drive/Gusto Performance/sources/preprocess/xgboost_forecast_finetuned.csv'

# Export the future_df to a CSV file
future_df.to_csv(output_path)

print(f"Future forecast exported to {output_path}")
fig.show()

print("\n--- Tuned Forecasting Visualization Complete ---")


--- Starting Tuned Forecasting Visualization ---
Dataset loaded successfully.
Data preparation complete.
Feature engineering complete.

--- Starting Hyperparameter Tuning for Random Forest ---
Fitting 5 folds for each of 54 candidates, totalling 270 fits

--- Hyperparameter Tuning Complete ---
Best Parameters Found:
{'max_depth': 20, 'max_features': 0.8, 'min_samples_leaf': 1, 'n_estimators': 500}

--- Generating Future Forecast with Tuned Model ---
Forecast generated for the next 90 days.

--- Creating Forecast Visualization ---
Future forecast exported to gdrive/My Drive/Gusto Performance/sources/preprocess/xgboost_forecast_finetuned.csv



--- Tuned Forecasting Visualization Complete ---


#random forest param grid

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import holidays
import plotly.graph_objects as go
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

print("--- Starting Tuned Random Forest Forecasting Script ---")

# --- 1. Data Loading and Preprocessing ---
print("\n--- Loading and Preprocessing Data ---")
try:
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    print("Dataset loaded successfully.")
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df = df[['date', 'answered_sessions']]
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)
    df = df.set_index('date').sort_index()
    print("Data preprocessing complete.")
except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    exit()

# --- 2. Feature Engineering ---
def create_features(df):
    df = df.copy()
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    return df

df = create_features(df)
print("Feature engineering complete.")

# --- 3. Hyperparameter Tuning with GridSearchCV ---
print("\n--- Starting Hyperparameter Tuning for Random Forest ---")
print("This may take a few minutes...")

TARGET = 'answered_sessions'
FEATURES = [col for col in df.columns if col != TARGET]
X_all = df[FEATURES]
y_all = df[TARGET]

# Define the grid of parameters to search through
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 0.8],
    'max_depth': [10, 20, None],
    'min_samples_leaf': [1, 2, 4]
}

# Use TimeSeriesSplit for cross-validation. This is crucial for time-series
# data as it respects the chronological order of observations.
#
tscv = TimeSeriesSplit(n_splits=5)

# Initialize the GridSearchCV object
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_mean_squared_error', # We optimize for the lowest error
    verbose=1
)

# Run the search on the entire dataset
grid_search.fit(X_all, y_all)

# Get the best model identified by the search
best_model = grid_search.best_estimator_
print("\n--- Hyperparameter Tuning Complete ---")
print("Best Parameters Found:")
print(grid_search.best_params_)


# --- 4. Generate 90-Day Future Forecast with the Tuned Model ---
print("\n--- Generating 90-Day Future Forecast ---")
last_date = df.index.max()
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=90, freq='D')
future_df = pd.DataFrame(index=future_dates)
future_df = create_features(future_df)

# Use the BEST model found by GridSearchCV to make predictions
future_predictions = best_model.predict(future_df[FEATURES])
future_df['forecast'] = future_predictions
print(f"Forecast generated for the next 90 days.")


# --- 5. Visualize the Historical Data and Forecast ---
print("\n--- Visualizing Full History and 90-Day Forecast ---")
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df[TARGET], mode='lines', name='Historical Data', line=dict(color='royalblue')))
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['forecast'], mode='lines', name='90-Day Forecast (Tuned)', line=dict(color='darkorange', dash='dash')))
fig.update_layout(
    title='Historical Call Volume and 90-Day Tuned Forecast',
    xaxis_title='Date', yaxis_title='Answered Sessions (Volume)',
    template='plotly_white',
    legend=dict(x=0.01, y=0.98)
)
# Define the path to save the CSV file
output_path = 'gdrive/My Drive/Gusto Performance/sources/preprocess/rf_forecast_finetuned.csv'

# Export the future_df to a CSV file
future_df.to_csv(output_path)

print(f"Future forecast exported to {output_path}")
fig.show()

print("\n--- Forecasting Script Complete ---")


--- Starting Tuned Random Forest Forecasting Script ---

--- Loading and Preprocessing Data ---
Dataset loaded successfully.
Data preprocessing complete.
Feature engineering complete.

--- Starting Hyperparameter Tuning for Random Forest ---
This may take a few minutes...
Fitting 5 folds for each of 54 candidates, totalling 270 fits

--- Hyperparameter Tuning Complete ---
Best Parameters Found:
{'max_depth': 20, 'max_features': 0.8, 'min_samples_leaf': 1, 'n_estimators': 500}

--- Generating 90-Day Future Forecast ---
Forecast generated for the next 90 days.

--- Visualizing Full History and 90-Day Forecast ---
Future forecast exported to gdrive/My Drive/Gusto Performance/sources/preprocess/rf_forecast_finetuned.csv



--- Forecasting Script Complete ---


#lightgbm

In [ ]:
import pandas as pd
import numpy as np
import holidays
import warnings
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import lightgbm as lgb
import plotly.graph_objects as go

# --- SUPPRESS WARNINGS ---
# This will hide the "No further splits with positive gain" messages.
warnings.filterwarnings('ignore', category=UserWarning)
print("--- Starting Master Forecasting Script (with Advanced Features & LightGBM) ---")

# --- Data Loading and Preprocessing ---
print("\n--- 1. Loading and Preprocessing Data ---")
try:
    df = pd.read_csv('gdrive/My Drive/Gusto Performance/sources/preprocess/Phone ASA + SL (table) - Hist.csv', delimiter='\t', header=0, encoding='utf-16')
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df = df[['date', 'answered_sessions']]
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    if df['answered_sessions'].dtype == 'object':
        df['answered_sessions'] = df['answered_sessions'].str.replace('"', '').str.replace(',', '').astype(int)
    df = df.set_index('date').sort_index()
    print("Data preprocessing complete.")
except FileNotFoundError:
    print("Error: The file 'Phone ASA + SL (table).csv' was not found.")
    exit()

# --- Advanced Feature Engineering ---
print("\n--- 2. Creating Advanced Time-Series Features ---")
def create_advanced_features(df):
    df = df.copy()
    # Date-based features
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    us_holidays = holidays.US()
    df['is_holiday'] = df.index.isin(us_holidays).astype(int)
    # Lag features
    df['lag_1'] = df['answered_sessions'].shift(1)
    df['lag_7'] = df['answered_sessions'].shift(7)
    df['lag_14'] = df['answered_sessions'].shift(14)
    # Rolling window features
    df['rolling_mean_7'] = df['answered_sessions'].shift(1).rolling(window=7).mean()
    df['rolling_std_7'] = df['answered_sessions'].shift(1).rolling(window=7).std()
    return df

df_featured = create_advanced_features(df)
df_featured = df_featured.dropna()
print("Advanced feature engineering complete.")

# --- Hyperparameter Tuning with GridSearchCV ---
print("\n--- 3. Tuning LightGBM Model ---")
TARGET = 'answered_sessions'
FEATURES = [col for col in df_featured.columns if col != TARGET]
X_all = df_featured[FEATURES]
y_all = df_featured[TARGET]

param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 40, 50],
    'max_depth': [-1],
    'reg_alpha': [0.1, 0.2 ,0.5, 0.7],
    'reg_lambda': [0.1, 0.2, 0.5, 0.7]
}
tscv = TimeSeriesSplit(n_splits=5)
grid_search = GridSearchCV(
    estimator=lgb.LGBMRegressor(random_state=42),
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_all, y_all)
best_model = grid_search.best_estimator_
print(f"\nBest Parameters Found: {grid_search.best_params_}")

# --- CORRECTED Recursive 90-Day Future Forecast ---
print("\n--- 4. Generating 90-Day Future Forecast (Recursive Method) ---")
future_predictions = []
# Start with the full history
history_df = df_featured.copy()
# Define future dates
future_dates = pd.date_range(history_df.index.max() + pd.Timedelta(days=1), periods=90, freq='D')

for date in future_dates:
    # Get the last row of features from our expanding history to predict the next day
    features_to_predict_with = history_df[FEATURES].iloc[-1:]

    # Predict the next day's volume
    prediction = best_model.predict(features_to_predict_with)[0]
    future_predictions.append(prediction)

    # Create a new row for the day we just predicted
    new_row = pd.DataFrame({'answered_sessions': [prediction]}, index=[date])

    # Append this new "day" to our history
    history_df = pd.concat([history_df, new_row])

    # Recalculate the features for the whole dataframe, which now includes our prediction.
    # This correctly generates the lag/rolling features for the *next* day to be predicted.
    history_df = create_advanced_features(history_df)

future_df = pd.DataFrame({'forecast': future_predictions}, index=future_dates)
print("Forecast generation complete.")

# --- Visualize the Historical Data and Forecast ---
print("\n--- 5. Visualizing Full History and 90-Day Forecast ---")
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df[TARGET], mode='lines', name='Historical Data', line=dict(color='royalblue')))
fig.add_trace(go.Scatter(x=future_df.index, y=future_df['forecast'], mode='lines', name='90-Day Forecast (Tuned LGBM)', line=dict(color='green', dash='dash')))
fig.update_layout(
    title='Historical Call Volume and 90-Day Tuned LightGBM Forecast',
    xaxis_title='Date', yaxis_title='Answered Sessions (Volume)',
    template='plotly_white',
    legend=dict(x=0.01, y=0.98)
)
# Define the path to save the CSV file
output_path = 'gdrive/My Drive/Gusto Performance/sources/preprocess/lgbm_forecast_finetuned.csv'

# Export the future_df to a CSV file
future_df.to_csv(output_path)

print(f"Future forecast exported to {output_path}")
fig.show()
print("\n--- Forecasting Script Complete ---")


--- Starting Master Forecasting Script (with Advanced Features & LightGBM) ---

--- 1. Loading and Preprocessing Data ---
Data preprocessing complete.

--- 2. Creating Advanced Time-Series Features ---
Advanced feature engineering complete.

--- 3. Tuning LightGBM Model ---
Fitting 5 folds for each of 144 candidates, totalling 720 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1611
[LightGBM] [Info] Number of data points in the train set: 1225, number of used features: 11
[LightGBM] [Info] Start training from score 1666.517551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin


--- Forecasting Script Complete ---
